In [ ]:
import random

class OdsekomaLinearnaFunkcija:
    """
    Razred za odsekoma linearne funkcije.
    """

    def __init__(self, tocke):
        """
        Konstruktor objekta.
        """
        tocke = dict(tocke) # naredimo nov slovar s točkami
        self.prelomi = sorted(tocke.keys()) # urejen seznam točk preloma
        self.vrednosti = [tocke[x] for x in self.prelomi] # seznam ustreznih vrednosti

    def __call__(self, x):
        """
        Evaluacija funkcije v točki x.
        """
        if x <= self.prelomi[0]: # levo od prve točke vrnemo prvo vrednost
            return self.vrednosti[0]
        elif x >= self.prelomi[-1]: # desno od zadnje točke vrnemo zadnjo vrednost
            return self.vrednosti[-1]
        i, j = 0, len(self.prelomi) - 1 # delamo bisekcijo
        while i < j - 1:
            h = (i + j) // 2
            if x < self.prelomi[h]:
                j = h
            else:
                i = h
        return ((self.prelomi[j] - x) * self.vrednosti[i] + (x - self.prelomi[i]) * self.vrednosti[j])  / (self.prelomi[j] - self.prelomi[i])

class TrikotnaFunkcija(OdsekomaLinearnaFunkcija):
    def __init__(self, a, b, c, h):
        """
        Konstruktor za trikotno funkcijo z vznožjem pri a in c ter vrhom pri b z vrednostjo h
        """
        assert a < b < c
        super().__init__({a: 0, b: h, c: 0})

    @staticmethod
    def generiraj(range_x=(0, 1), range_y=(0, 100)):
        """
        Generator funkcij pri podanih rangih vrednosti.
        """
        a, b, c = sorted(random.uniform(*range_x) for _ in range(3))
        h = random.uniform(*range_y)
        return TrikotnaFunkcija(a, b, c, h)


################################################################################################################################################
# Nastavitve programa 
# Tukaj nastavimo število trikotnih funkcij(n) in število trikotnih funkcij v podmnožici(z), z < n
# Nastavimo zaprt interval [a,c], a < c

n = 10
z = 7
a = 0
c = 1
d = (c-a)/2

if n <= 0:
    print("Napaka: število trikotnih funkcij mora biti pozitivno")
elif n < z:
    print("Napaka: števlo vseh trikotnih funkcij mora biti večje kot število v podmnožici")
elif a >= c :
    print("Napaka: upoštevaj pogoj a < b")
else:
    print("Uspešno nastavljene nastavitve")


################################################################################################################################################

seznam_funkcij = [TrikotnaFunkcija.generiraj(range_x=(-2, 2), range_y=(-10, 10)) for i in range(n)]
a, c = -1, 1 # gledamo vrednosti v intervalu [a, c]
seznam_tock = sorted(set(x for f in seznam_funkcij for x in f.prelomi if a <= x <= b).union({a, c}))


def vsota_trikotnih_funkcij(seznam_funkcij, seznam_tock, z):
    k = len(seznam_tock)
    r = len(seznam_funkcij)

    p = MixedIntegerLinearProgram(maximization=False)
    v = p.new_variable(binary=True)
    m = p.new_variable(integer=False, nonnegative=True)
    p.set_objective(m[1]-m[0])

    for x in seznam_tock:
        #Izračun prve vsote(f(x)*v)
        sum_ = sum(seznam_funkcij[j](x) * v[j] for j in range(r))
        f'{sum_=}'
        f'{sum_ - m[1]}'
        f'{sum_ - m[0]}'
        p.add_constraint(sum_ - m[1], max=0)
        p.add_constraint(sum_ - m[0], min=0)


    sum2_ = sum(v[j] for j in range(z))
        #Izračun druge vsote vsote(v_j)
    f'{sum2_=}'
    f'{v[0]=}'
    f'{z=}'
    p.add_constraint(sum2_, min=1)
    p.add_constraint(sum2_, max=z)

    vsota = p.solve()
    M, m = p.get_values(m[1]), p.get_values(m[0])
    f'{M=}'
    f'{m=}'
    v_resitve = p.get_values(v)
    f'{p.get_values(v)}'

    vkljucene_funkcije = [seznam_funkcij[j] for j in v_resitve if v_resitve[j]==1]
        #Trikotne funkcije v naboru

    return [vsota, vkljucene_funkcije]


vsota_trikotnih_funkcij(seznam_funkcij, seznam_tock, 7)

################################################################################################################################################

################################################################################################################################################

import time

    #Spreminanje časa izvajanja v odvisnosti od n(število trikotnih funkcij)

def cas_izvajanja(seznam_tock, seznam_funkcij,n):
    casi = []
    for i in range(1,n+1):
        zacetek = time.time()
        vsota_trikotnih_funkcij(seznam_funkcij, seznam_tock)
        konec = time.time() - zacetek
        casi.append((konec))
    return casi

#cas_izvajanja(seznam_tock, seznam_funkcij, 10)

